In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Data Preparation (same as before)
df = pd.read_csv('C:\\Users\\ASUS\\Downloads\\Code for VAE-BiLSTM Paper\\DailyBeijingClimate.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

features = ['temperature', 'humidity', 'wind_speed', 'power']
df = df[features]

scalers = {}
for column in df.columns:
    scaler = MinMaxScaler()
    df[column] = scaler.fit_transform(df[[column]])
    scalers['scaler_' + column] = scaler

n_past = 10
n_future = 5
n_features = len(features)

X = []
y = []
for i in range(n_past, len(df) - n_future + 1):
    X.append(df.iloc[i - n_past:i].values)
    y.append(df.iloc[i:i + n_future].values)

X = np.array(X)
y = np.array(y)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 2. LSTM Model (Simplified - No VAE)
model_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
lstm_1 = tf.keras.layers.LSTM(8, return_sequences=True)(model_inputs)  # Increased LSTM units
lstm_2 = tf.keras.layers.LSTM(4, return_sequences=False)(lstm_1) # Changed return_sequences to False
model_outputs = tf.keras.layers.Dense(n_features * n_future)(lstm_2) # Removed TimeDistributed and adjusted output shape
model_outputs = tf.keras.layers.Reshape([n_future, n_features])(model_outputs)  # Reshape output to match y_train

model = tf.keras.models.Model(inputs=model_inputs, outputs=model_outputs)

# 3. Compilation and Training (same as before)
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber()) # Using Huber loss
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=4, verbose=1, callbacks=[reduce_lr, early_stopping])

# 4. Prediction and Inverse Scaling (same as before)
pred = model.predict(X_test)

for index, i in enumerate(features):
    scaler = scalers['scaler_' + i]
    pred[:, :, index] = scaler.inverse_transform(pred[:, :, index])
    y_train[:, :, index] = scaler.inverse_transform(y_train[:, :, index])
    y_test[:, :, index] = scaler.inverse_transform(y_test[:, :, index])

# 5. Evaluation (same as before)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true_all = y_test.flatten()
y_pred_all = pred.flatten()

mae = mean_absolute_error(y_true_all, y_pred_all)
mse = mean_squared_error(y_true_all, y_pred_all)
rmse = np.sqrt(mse)
r2 = r2_score(y_true_all, y_pred_all)

results_df = pd.DataFrame({
    "Model": ["LSTM"],
    "MAE": [mae],
    "MSE": [mse],
    "RMSE": [rmse],
    "R2 Score": [r2]
})

print(results_df)

y_test_2d = y_test.reshape(-1, y_test.shape[-1])
pred_2d = pred.reshape(-1, pred.shape[-1])

y_var = np.var(y_test_2d)
normalized_mse = mean_squared_error(y_test_2d, pred_2d) / y_var

print("Normalized MSE LSTM:", normalized_mse)

# Predict next 10 days (same as before)
X_test_last = X_test[-1:].reshape(1, n_past, n_features)
pred_next_10_days = model.predict(X_test_last)

for index, i in enumerate(features):
    scaler = scalers['scaler_' + i]
    pred_next_10_days[:, :, index] = scaler.inverse_transform(pred_next_10_days[:, :, index])

print("Predicted next 10 days (LSTM):", pred_next_10_days)

Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0552 - val_loss: 0.0110 - learning_rate: 0.0010
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0125 - val_loss: 0.0084 - learning_rate: 9.0000e-04
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0116 - val_loss: 0.0079 - learning_rate: 8.1000e-04
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0106 - val_loss: 0.0072 - learning_rate: 7.2900e-04
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0101 - val_loss: 0.0064 - learning_rate: 6.5610e-04
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0085 - val_loss: 0.0056 - learning_rate: 5.9049e-04
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0067 - val_loss: 0.0054 - learning_rate: 5.3144e-04
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0059 - val_loss: 0.0049 - learning_rate: 4.7830e-04
Epoch 9/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0055 - val_loss: 0.0048 - lear